## 多进程

### Fork

Linux系统中使用fork()函数复制父进程，fork()函数返回两次。一次是父进程，一次是子进程。子进程进程永远返回0，其通过getppid()得到父进程的进程号。

In [1]:
import os

In [2]:
print("start process {}".format(os.getpid()))

start process 2540


In [3]:
pid = os.fork()
if pid == 0:
    print("I am child process {} of father process {}".format(os.getpid(), os.getppid()))
else:
    print("I {} start a child process {}".format(os.getpid(), pid))

I 2540 start a child process 2689
I am child process 2689 of father process 2540
ERROR! Session/line number was not unique in database. History logging moved to new session 199


### multiprocessing

In [6]:
from multiprocessing import Process

In [5]:
def run_func(name):
    print("Process {} run_func({})".format(os.getpid(), name))

In [20]:
print("Parent process {}".format(os.getpid()))
# args=('test',)  => 只有一个元素的元组
# args=('test')   => args是一个字符串，值为'test'
p = Process(target=run_func, args=('test',))
print("Child process start...")
p.start()
p.join()
print("Child process end")

Parent process 2540
Child process start...
Process 3211 run_func(test)
Child process end


### 进程池

In [15]:
! cat process_pool.py

from multiprocessing import Pool
import os, time, random

NUM_PROCESSINGS = 3

def task(name):
	start = time.time()
	print("Child process {} run task {}".format(os.getpid(), name))
	time.sleep(random.random() * 3)
	end = time.time()
	print("Child process {} run task {} for {:.6f} second".format(os.getpid(), name, (end-start)))

if __name__ == '__main__':
	print("Parent process {}".format(os.getpid()))

	# 每次最多运行NUM_PROCESSINGS个进程
	pool = Pool(NUM_PROCESSINGS)
	for x in range(NUM_PROCESSINGS+1):
		pool.apply_async(task, args=(x,))
	print("Waiting all subprocessings finish...")
	pool.close()
	pool.join()
	print("All subprocessings finished.")


### 子进程

创建一个外部子进程，并且控制其输入和输出。 在jupyter notebook中无法看到命令的输出结果，但是执行.py文件可以在终端看到命令的输出结果。

In [11]:
print('$ date')
r = subprocess.call('date')
print('Exit code:', r)

$ date
Exit code: 0


In [16]:
! cat sub_process.py

import subprocess

print('$ date')
r = subprocess.call(['date'])
print('Exit code:', r)



## 子进程需要输入，可以使用communicate方法
print('$ nslookup')
# 首先输入: nsloopup命令
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# 然后依次每行输入：
# set q=mx
# python.org
# exit
output, error = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print("Exit code {}".format(p.returncode))


### 进程间通信

## 线程

In [19]:
import threading

In [22]:
def task(name):
    print("Thread {} is running...".format(threading.current_thread().name))
    for x in range(3):
        print("loop {}: {}".format(x, name))
    print("Thread {} finished.".format(threading.current_thread().name))

In [23]:
print("Thread {} is running".format(threading.current_thread().name))
t = threading.Thread(target=task, args=('test',), name='SubThread')
t.start()
t.join()
print("Thread {} is finish".format(threading.current_thread().name))

Thread MainThread is running
Thread SubThread is running...
loop 0: test
loop 1: test
loop 2: test
Thread SubThread finished.
Thread MainThread is finish


多线程修改全局变量可能带来数据不一致的问题，因此必须试用锁

In [66]:
def change_balance(n):
    global balance
    balance += n
    balance -= n

In [67]:
def task_no_lock():
    for x in range(100000):
        change_balance(x)

In [72]:
balance = 0
t1 = threading.Thread(target=task_no_lock)
t2 = threading.Thread(target=task_no_lock)
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

76598


In [61]:
def task():
    for x in range(100000):
        # 1. 先尝试获取锁
        lock.acquire()
        try:
            # 2. 修改
            change_balance(x)
        finally:
            # 3. 释放锁
            lock.release()

In [80]:
balance = 0
lock = threading.Lock()

t1 = threading.Thread(target=task)
t2 = threading.Thread(target=task)
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


### ThreadLocal

In [81]:
import threading

In [88]:
# 记录着每个线程特有的线程全局变量
local_thread = threading.local()

In [83]:
def process_student():
    student = local_thread.student
    print("Student {} in thread {}".format(student, threading.current_thread().name))

In [86]:
def process_thread(name):
    local_thread.student = name
    process_student()

In [87]:
t1 = threading.Thread(target=process_thread, args=('Alice',), name='Thread1')
t2 = threading.Thread(target=process_thread, args=('Sam',), name='Thread2')
t1.start()
t2.start()
t1.join()
t2.join()

Student Alice in thread Thread1
Student Sam in thread Thread2


ThreadLocal最常用的地方就是为每个线程绑定一个数据库连接，HTTP请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。

计算密集型任务的特点是要进行大量的计算，消耗CPU资源，比如计算圆周率、对视频进行高清解码等等，全靠CPU的运算能力。这种计算密集型任务虽然也可以用多任务完成，但是任务越多，花在任务切换的时间就越多，CPU执行任务的效率就越低，所以，要最高效地利用CPU，计算密集型任务同时进行的数量应当等于CPU的核心数。

**计算密集型任务由于主要消耗CPU资源**，因此，代码运行效率至关重要。Python这样的脚本语言运行效率很低，完全不适合计算密集型任务。**对于计算密集型任务，最好用C语言编写**。

第二种任务的类型是IO密集型，**涉及到网络、磁盘IO的任务都是IO密集型任务**，这类任务的特点是CPU消耗很少，任务的大部分时间都在等待IO操作完成（因为IO的速度远远低于CPU和内存的速度）。对于IO密集型任务，任务越多，CPU效率越高，但也有一个限度。常见的大部分任务都是IO密集型任务，比如Web应用。

IO密集型任务执行期间，99%的时间都花在IO上，花在CPU上的时间很少，因此，用运行速度极快的C语言替换用Python这样运行速度极低的脚本语言，完全无法提升运行效率。**对于IO密集型任务，最合适的语言就是开发效率最高**（代码量最少）的语言，脚本语言是首选，C语言最差。

现代操作系统对IO操作已经做了巨大的改进，最大的特点就是支持异步IO。如果充分利用操作系统提供的异步IO支持，就可以用单进程单线程模型来执行多任务，这种全新的模型称为事件驱动模型，Nginx就是支持异步IO的Web服务器，它在单核CPU上采用单进程模型就可以高效地支持多任务。在多核CPU上，可以运行多个进程（数量与CPU核心数相同），充分利用多核CPU。由于系统总的进程数量十分有限，因此操作系统调度非常高效。用异步IO编程模型来实现多任务是一个主要的趋势。

对应到Python语言，**单线程的异步编程模型称为协程**，**有了协程的支持，就可以基于事件驱动编写高效的多任务程序**。我们会在后面讨论如何编写协程。